# <center>Big Data &ndash; Exercises</center>
## <center>Spring 2022 &ndash; Week 9 &ndash; ETH Zurich</center>
## <center>Spark Dataframes and Spark SQL, Moodle exercise</center>

# Preparation for the moodle exercise in Spark

In this jupyter notebook we are going to make the preprocessing part of the dataset that is going to be used in the graded exercise of this week.
It will be the same language game dataset as in exercise08.

1. Change to `exercise09` repository

2. Start docker <br>
```docker-compose up -d```

3. After docker finishes downloading the images, you should be able to start the jupyter notebook by copying the following URL to your browser <br>
```http://localhost:8888/lab```

4. Getting the data:
Follow the procedure that is described below. The dataset can be found here: https://cloud.inf.ethz.ch/s/a8FoHew6dHKGYKK/download/confusion20140302.tbz2 

More specifically do the following:
- download the data      :<br> ```wget https://cloud.inf.ethz.ch/s/a8FoHew6dHKGYKK/download/confusion20140302.tbz2```
- extract the data       :<br> ```tar -jxvf confusion20140302.tbz2```


## More Info about the data
You can find more information about the dataset (as well as the schema and examples) in this link: https://quietlyamused.org/blog/2014/03/12/language-confusion/

## Instructions:

In every query we ask you for three quantities: the query itself, the result of the query, the time it took you to write the query. Note that the time part of every question is optional and not graded. In order to make easier the time recording we created two functions that do it automatically. Run the cell below in order to import the functions into the current notebook. Then before each query we will have a ```start_exercise()``` cell that you have to run in order to start time recording. After you have finished your query and you are sure about the answer run the ```finish_exercise()``` one to get the time measurement. 

#### <b>For the assignments we only use the first 50k lines of the dataset, `dataset_50k = dataset.limit(50000)`. </b>

In [1]:
import time

def start_exercise():
    global last
    last = time.time()
    
def finish_exercise():
    global last
    print("This exercise took {0}s".format(int(time.time()-last)))

## <center>1. Spark Dataframes</center>

Write queries for the same questions as last week, but this time using Spark Dataframes operations (the data loading will take a couple minutes).

### 1.0. Data preprocessing

In [2]:
import json
from pyspark.sql import SparkSession
from pyspark import SparkConf

spark = SparkSession.builder.master('local').getOrCreate()
sc = spark.sparkContext

path = "confusion-2014-03-02/confusion-2014-03-02.json"
dataset = spark.read.json(path).cache()

In [3]:
#test it out
dataset.limit(3).show()

+--------------------+-------+----------+---------+--------------------+---------+
|             choices|country|      date|    guess|              sample|   target|
+--------------------+-------+----------+---------+--------------------+---------+
|[Maori, Mandarin,...|     AU|2013-08-19|Norwegian|48f9c924e0d98c959...|Norwegian|
|[Danish, Dinka, K...|     AU|2013-08-19|    Dinka|af5e8f27cef9e689a...|    Dinka|
|[German, Hungaria...|     AU|2013-08-19|  Turkish|509c36eb58dbce009...|   Samoan|
+--------------------+-------+----------+---------+--------------------+---------+



In [4]:
dataset_50k = dataset.limit(50000)

## Assignment 1
Find all games such that the guessed language is correct (=target), and such that this language is Spanish. What is the length of the resulting sequence?

In [5]:
start_exercise()

In [34]:
# your code here
#dataset_50k.filter("target = 'Spanish' and guess = 'Spanish'").count()
a = dataset_50k.filter(dataset_50k["target"] == 'Spanish')
b = a.filter(a["guess"] == 'Spanish')
b.count()

1010

In [7]:
finish_exercise()

This exercise took 65s


## Assignment 2
Find the number of all distinct values of the guessed languages (i.e. the guess field). What is the length of the resulting sequence?

In [8]:
start_exercise()

In [9]:
dataset_50k.select("guess").distinct().count()

68

In [10]:
finish_exercise()

This exercise took 1s


## Assignment 3
Return the top three games where the guessed language is incorrect ($\ne$target) ordered by country (ascending), then target language (ascending), then date (ascending). What is the sample id of the 3rd item in the list? 

Enter it without quotes, for example 48f9c924e0d98c959d8a6f1862b3ce9a

In [11]:
start_exercise()

In [38]:
from pyspark.sql.functions import asc

dataset_50k.select("sample").filter("guess != target").orderBy(
    dataset_50k["country"].asc(), dataset_50k["target"].asc(), dataset_50k["date"].asc()
).limit(3).collect()

[Row(sample='ccebbc271377f7173e39564c74901f7c'),
 Row(sample='7efab6f0c61694ba6e8aa72843d1bf15'),
 Row(sample='3520fdcdf2658685928fc37e72ab2134')]

In [13]:
finish_exercise()

This exercise took 1s


## Assignment 4
Aggregate all games by guessed and target language, counting the number of guessing games that were done for each pair (guess, target). How many times has Dutch been mistaken for Norwegian (i.e. Dutch was the true answer)?

In [14]:
start_exercise()

In [39]:
from pyspark.sql.functions import desc

dataset_50k.groupBy(["guess", "target"]).count().filter("target == 'Norwegian' and guess = 'Dutch'").select("count").collect()

[Row(guess='Dutch', target='Norwegian', count=9)]

In [16]:
finish_exercise()

This exercise took 1s


## Assignment 5
Among all the games where the guess was correct (=target), what is the percentage of cases where the second choice (among the array of possible answers) was the target?

Please write the fraction rounding to 4 decimals (eg. 0.3323)

In [17]:
start_exercise()

In [43]:
dataset_50k.filter("choices[1] = target and target = guess").count() / float(dataset_50k.count())

0.2696

In [19]:
finish_exercise()

This exercise took 2s


## Assignment 6
How many games in France (country=FR) were played on the last day?

In [20]:
start_exercise()

In [21]:
max_date = dataset_50k.agg({"date": "max"}).collect()[0]["max(date)"]
dataset_50k.filter(dataset_50k["date"] == max_date).filter(dataset_50k["country"] == "FR").count()

93

In [22]:
finish_exercise()

This exercise took 1s


## <center>2. Spark SQL</center>

Write Spark SQL queries for the same questions as earlier.

### 2.0. Data preprocessing

In [23]:
!pip install sparksql-magic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 3.8 MB/s eta 0:00:00a 0:00:01


In [24]:
%load_ext sparksql_magic

In [25]:
path = "confusion-2014-03-02/confusion-2014-03-02.json"
dataset_50k = spark.read.json(path).cache().limit(50000)
dataset_50k.createOrReplaceTempView("dataset_50k")

In [26]:
%%sparksql
-- test it out
SELECT *
FROM dataset_50k
LIMIT 3

choices,country,date,guess,sample,target
"['Maori', 'Mandarin', 'Norwegian', 'Tongan']",AU,2013-08-19,Norwegian,48f9c924e0d98c959d8a6f1862b3ce9a,Norwegian
"['Danish', 'Dinka', 'Khmer', 'Lao']",AU,2013-08-19,Dinka,af5e8f27cef9e689a070b8814dcc02c3,Dinka
"['German', 'Hungarian', 'Samoan', 'Turkish']",AU,2013-08-19,Turkish,509c36eb58dbce009ccf93f375358d53,Samoan


## Assignment 1
Find all games such that the guessed language is correct (=target), and such that this language is Spanish. What is the length of the resulting sequence?

In [27]:
start_exercise()

In [44]:
%%sparksql
SELECT count(*) FROM dataset_50k
WHERE target == "Spanish" 
AND target == guess


count(1)
1010


In [ ]:
finish_exercise()

## Assignment 2
Find the number of all distinct values of the guessed languages (i.e. the guess field). What is the length of the resulting sequence?

In [ ]:
start_exercise()

In [45]:
%%sparksql
SELECT count(distinct(guess))
FROM dataset_50k

count(DISTINCT guess)
68


In [ ]:
finish_exercise()

## Assignment 3
Return the top three games where the guessed language is incorrect ($\ne$target) ordered by country (ascending), then target language (ascending), then date (ascending). What is the sample id of the 3rd item in the list? 

Enter it without quotes, for example 48f9c924e0d98c959d8a6f1862b3ce9a

In [ ]:
start_exercise()

In [55]:
%%sparksql
SELECT sample
FROM dataset_50k as d
WHERE guess != target
ORDER BY country asc, target asc, d.date asc
LIMIT 3

sample
ccebbc271377f7173e39564c74901f7c
7efab6f0c61694ba6e8aa72843d1bf15
3520fdcdf2658685928fc37e72ab2134


In [ ]:
finish_exercise()

## Assignment 4
Aggregate all games by guessed and target language, counting the number of guessing games that were done for each pair (guess, target). How many times has Dutch been mistaken for Norwegian (i.e. Dutch was the true answer)?

In [ ]:
start_exercise()

In [53]:
%%sparksql
SELECT count(guess)
FROM dataset_50k 
GROUP BY country, target
ORDER BY count(guess) desc
LIMIT 3

sample
5cd22cbf16be1a93d304d589431c3ec3
65bf2aceadf2520f36a5e548a0471fcf
bf0f1eec2e9f147aba20ea9b3e9564bb


In [ ]:
finish_exercise()

## Assignment 5
Among all the games where the guess was correct (=target), what is the percentage of cases where the second choice (among the array of possible answers) was the target?

Please write the fraction rounding to 4 decimals (eg. 0.3323)

In [ ]:
start_exercise()

In [60]:
%%sparksql
SELECT count(*) / 
(SELECT count(*) as c
FROM dataset_50k)
FROM dataset_50k
WHERE choices[1] = target AND target = guess

(count(1) / scalarsubquery())
0.2696


In [ ]:
finish_exercise()

## Assignment 6
How many games in France (country=FR) were played on the last day?

In [ ]:
start_exercise()

In [58]:
%%sparksql
SELECT count(*) 
FROM dataset_50k
WHERE date = 
(SELECT max(date) FROM dataset_50k)
AND country = "FR"

count(1)
93


In [ ]:
finish_exercise()